# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

* Litesh

In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [13]:
# --- store the filepath in a variable ---
file = "../output/cities.csv"

# --- read the csv file and create a dataframe ---
city_weather_df = pd.DataFrame(pd.read_csv(file, index_col="City"))

# --- display the dataframe ---
city_weather_df


,City.1,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind speed,Country,Date
City,,,,,,,,,
0,srednekolymsk,67.4500,153.6833,48.76,64,100,18.12,RU,1623808227
1,canutama,-6.5339,-64.3831,71.53,98,99,2.84,BR,1623808228
2,kiunga,-6.1219,141.2906,83.19,75,79,2.98,PG,1623808229
3,poum,-20.2333,164.0167,75.85,81,31,8.16,NC,1623808229
4,bambous virieux,-20.3428,57.7575,70.21,100,75,4.61,MU,1623808230
...,...,...,...,...,...,...,...,...,...
557,keti bandar,24.1447,67.4497,83.37,82,100,13.29,PK,1623808643
558,sao joao da barra,-21.6403,-41.0511,67.51,72,0,4.59,BR,1623808399
559,elizabeth city,36.2946,-76.2511,76.01,87,18,2.64,US,1623808645


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [34]:
#Configure gmaps
gmaps.configure(api_key=g_key)

#Get locations and humidity

locations = city_weather_df[["Lat", "Lng"]]
weights = city_weather_df["Humidity"]

# Format heatmap
layout = {
    'width': "100%",
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px',

}
#Formattting
fig = gmaps.figure(layout=figure_layout, center=(20.0, 5.0), zoom_level=1.9)
heatmap_layer = gmaps.heatmap_layer(locations, weights=weights, dissipating=False, max_intensity=100,
                                 point_radius = 1)
#Add the heatmap layer
fig.add_layer(heatmap_layer)
#display heatmap
fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', padding='1px', width='100%'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [17]:
# Dataframe to hold ideal weather conditions
ideal_cities = cities_data.loc[(city_weather_df["Max Temp"] > 70) & 
                               (city_weather_df["Max Temp"] < 80) & 
                               (city_weather_df["Wind speed"] < 10) & 
                               (city_weather_df["Cloudiness"] == 0) &
                               #drop NA
                               (city_weather_df["Humidity"] < 75), :].reset_index(drop=True).dropna()
ideal_cities

,City.1,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind speed,Country,Date
0,awbari,26.5921,12.7805,77.68,15,0,5.75,LY,1623808305
1,vostok,46.4856,135.8833,77.81,40,0,4.38,RU,1623808390
2,sakakah,29.9697,40.2064,78.37,25,0,8.03,SA,1623808440
3,urumqi,43.8010,87.6005,70.75,23,0,8.95,CN,1623808526
4,puertollano,38.6871,-4.1073,73.96,74,0,5.70,ES,1623808463
5,shunyi,40.1250,116.6453,73.83,64,0,6.71,CN,1623808556
6,villanueva de la serena,38.9766,-5.7974,71.83,68,0,2.19,ES,1623808573
7,zhanaozen,43.3000,52.8000,79.90,13,0,8.66,KZ,1623808616
8,waddan,29.1614,16.1390,72.59,46,0,7.70,LY,1623808633


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [19]:
# Dataframe hotel_df 
hotel_df = ideal_cities[["City.1", "Country", "Lat", "Lng"]].copy()

# --- Add a "Hotel Name" column to the DataFrame with null values ---
hotel_df["Hotel Name"] = np.nan
hotel_df

,City.1,Country,Lat,Lng,Hotel Name
0,awbari,LY,26.5921,12.7805,NaN
1,vostok,RU,46.4856,135.8833,NaN
2,sakakah,SA,29.9697,40.2064,NaN
3,urumqi,CN,43.8010,87.6005,NaN
4,puertollano,ES,38.6871,-4.1073,NaN
5,shunyi,CN,40.1250,116.6453,NaN
6,villanueva de la serena,ES,38.9766,-5.7974,NaN
7,zhanaozen,KZ,43.3000,52.8000,NaN
8,waddan,LY,29.1614,16.1390,NaN


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
